In [1]:
# dataset.py
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset
import torchvision.transforms as T

class ChocoDataset(Dataset):
    def __init__(self, csv_file, img_dir, transform=None):
        self.labels_df = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transform or T.Compose([
            T.Resize((224, 224)),
            T.ToTensor()
        ])
        self.image_ids = self.labels_df['id'].astype(str)
        self.labels = self.labels_df.drop(columns=['id']).values.astype(float)

    def __len__(self):
        return len(self.image_ids)

    def __getitem__(self, idx):
        img_id = self.image_ids.iloc[idx]
        img_path = os.path.join(self.img_dir, f"L{img_id}.JPG")
        image = Image.open(img_path).convert("RGB")
        image = self.transform(image)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return image, label


In [2]:
# model.py
import torch.nn as nn
import torchvision.models as models

class ChocoNet(nn.Module):
    def __init__(self, num_classes=13):
        super().__init__()
        self.backbone = models.resnet18(pretrained=False)
        self.backbone.fc = nn.Sequential(
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
            nn.ReLU()  # 保证非负输出
        )

    def forward(self, x):
        return self.backbone(x)


In [ ]:
# train.py
import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from dataset import ChocoDataset
from model import ChocoNet
import os

# 设置路径
csv_path = 'dataset_project_iapr2025/train/train.csv'
img_dir = 'dataset_project_iapr2025/train/'

# 数据加载
dataset = ChocoDataset(csv_file=csv_path, img_dir=img_dir)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# 模型初始化
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ChocoNet(num_classes=13).to(device)

# 损失函数 & 优化器
criterion = torch.nn.MSELoss()
optimizer = Adam(model.parameters(), lr=1e-3)

# 训练过程
epochs = 30
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for images, labels in dataloader:
        images, labels = images.to(device), labels.to(device)

        preds = model(images)
        loss = criterion(preds, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs} - Loss: {total_loss/len(dataloader):.4f}")

# 保存模型
os.makedirs('weights', exist_ok=True)
torch.save(model.state_dict(), 'weights/choco_model.pth')
